In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import numpy as np
import time

데이터 불러오기

In [2]:
with open('2016_filtered_review_part.txt', encoding='utf-8') as f:
    docs = [doc.strip().split('\t\t') for doc in f]
    docs = [(doc[1], int(doc[2])) for doc in docs if len(doc) == 3]
    # 각 행에서 두 번째와 세 번째 칼럼을 읽음
    texts, scores = zip(*docs)
    # 둘을 분리해서 별도의 list 변수로 저장

평점 정보를 이용해서 종속변수 레이블링 하기

In [3]:
filtered_texts = []
filtered_labels = []

for text, score in zip(texts, scores):
    if 4 < score < 8:
        continue
        
    # 평점 기준으로 문서에 label을 부여
    # 1 ~ 4 -> 부정, 0
    # 8 ~ 10 -> 긍정, 1
    filtered_texts.append(text)
    filtered_labels.append(1 if score >= 8 else 0) # 조건부 표현 (conditional expression) 사용

In [4]:
# 불균형 문제
print('전체 영화평의 수: {}'.format(len(filtered_labels)))
print('긍정 영화평의 수: {}'.format(sum(filtered_labels)))
print('긍정 영화평의 비율: {0:.2f}'.format(sum(filtered_labels)/len(filtered_labels)))

전체 영화평의 수: 19513
긍정 영화평의 수: 17890
긍정 영화평의 비율: 0.92


In [5]:
# To split the data into training and test datasets
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.1, random_state=42)

In [6]:
train_texts[0]

'영화 괜찮 재미있'

# 1. L1 규제화 사용해 보기

### 1.1 TF 정보 사용해 보기

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer() 
tf_train_features = tf_vectorizer.fit_transform(train_texts) 
tf_test_features = tf_vectorizer.transform(test_texts)

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
lr_tf_l1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=10000) 

In [10]:
start = time.time()
lr_tf_l1.fit(tf_train_features, train_labels) # 학습
end = time.time()
elapsed = end - start 
print('\nLearning time: {0:.3f} seconds'.format(elapsed))


Learning time: 3.211 seconds


In [11]:
pred_labels_tf_l1 = lr_tf_l1.predict(tf_test_features) # 예측

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
accuracy_score(test_labels, pred_labels_tf_l1)

0.9385245901639344

긍정과 부정의 역할을 하는 단어 출력해 보기

In [14]:
# 단어 사전 만들기
words_dict = {}
for word, index in tf_vectorizer.vocabulary_.items():
    words_dict[index]=word

In [15]:
# Get coefficients of the model
coefficients = lr_tf_l1.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'


K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("\n부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
재밌 (1.515)
재미있 (1.427)
최고 (1.269)
역시 (0.926)
즐겁 (0.888)
영상미 (0.731)
도르마무 (0.611)
다음 (0.546)
추천 (0.499)
아이 (0.489)
영상 (0.466)
아쉽 (0.431)
마블 (0.382)
연기 (0.331)
스트레인 (0.283)
새롭 (0.255)
아이맥스 (0.220)
효과 (0.214)
약간 (0.208)
기대 (0.184)
베네딕트 (0.146)
베니 (0.146)
망토 (0.142)
닥터스 (0.131)
생각 (0.113)
거북이 (0.101)
멋지 (0.058)
괜찮 (0.032)
조금 (0.006)
독특 (0.001)

부정 단어 상위 30 개
처음 (-0.258)
진심 (-0.268)
나오 (-0.273)
기분 (-0.297)
영화 (-0.312)
너무 (-0.333)
개연 (-0.399)
졸리 (-0.443)
초딩 (-0.487)
핵노잼 (-0.521)
음악 (-0.541)
스토리 (-0.556)
그냥 (-0.569)
노래 (-0.601)
손예진 (-0.642)
내용 (-0.738)
정신 (-0.739)
평점 (-0.805)
실망 (-0.968)
쓰레기 (-0.971)
유치 (-1.089)
짜증 (-1.189)
낭비 (-1.296)
감독 (-1.344)
별로 (-1.405)
알바 (-1.847)
아깝 (-1.908)
재미없 (-2.451)
노잼 (-2.717)
최악 (-2.902)


### 1.2 TF-IDF 정보 사용해 보기

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer() 
tfidf_train_features = tfidf_vectorizer.fit_transform(train_texts) 
tfidf_test_features = tfidf_vectorizer.transform(test_texts)

In [17]:
lr_tfidf_l1 = LogisticRegression(C=0.1, penalty='l1', solver='saga', max_iter=10000) 
lr_tfidf_l1.fit(tfidf_train_features, train_labels) # 학습
pred_labels_tfidf_l1 = lr_tfidf_l1.predict(tfidf_test_features)

In [18]:
accuracy_score(test_labels, pred_labels_tfidf_l1)

0.9339139344262295

상위 긍, 부정 단어 출력해 보기

In [19]:
coefficients = lr_tfidf_l1.coef_.tolist()
sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)

K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
재밌 (3.789)
재미있 (2.568)
최고 (1.666)
마블 (1.422)
역시 (1.057)
도르마무 (0.910)
영상미 (0.563)
즐겁 (0.369)
아이 (0.136)
기대 (0.127)
영상 (0.046)
1펀 (0.000)
2디 (0.000)
2화 (0.000)
3d (0.000)
3디 (0.000)
3베니마블 (0.000)
4d (0.000)
4디 (0.000)
d강추 (0.000)
oo노잼 (0.000)
ㄱ저냥 (0.000)
ㄹ혜 (0.000)
ㅈ노잼 (0.000)
ㅈ루 (0.000)
ㅜ나빡 (0.000)
ㅜ컴버배치 (0.000)
ㅡ스리디 (0.000)
ㅡ재밋는 (0.000)
日고미 (0.000)
부정 단어 상위 30 개
히트 (0.000)
히트다히트 (0.000)
히트작 (0.000)
히피 (0.000)
히헿 (0.000)
히히 (0.000)
히히히헤헤헤히힣 (0.000)
힌트 (0.000)
힐로 (0.000)
힐링 (0.000)
힘내 (0.000)
힘들 (0.000)
힘쓰 (0.000)
힘없이 (0.000)
힙합 (0.000)
너무 (-0.041)
음악 (-0.190)
그냥 (-0.560)
스토리 (-0.672)
영화 (-1.036)
내용 (-1.238)
손예진 (-1.374)
별로 (-2.278)
노잼 (-2.861)
평점 (-2.918)
아깝 (-3.321)
알바 (-3.331)
감독 (-3.809)
재미없 (-4.715)
최악 (-6.185)


# L2 규제화 사용해 보기

### TF 정보 사용하기

In [20]:
lr_tf_l2 = LogisticRegression(C=0.1, penalty='l2', solver='saga', max_iter=10000)

In [21]:
start = time.time()
lr_tf_l2.fit(tf_train_features, train_labels) # 학습
end = time.time()
elapsed = end - start 
print('\nLearning time: {0:.3f} seconds'.format(elapsed))


Learning time: 0.736 seconds


In [22]:
pred_labels_tf_l2 = lr_tf_l2.predict(tf_test_features) # 예측

In [23]:
accuracy_score(test_labels, pred_labels_tf_l2)

0.9395491803278688

In [24]:
# Get coefficients of the model
coefficients = lr_tf_l2.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'


K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("\n부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
재밌 (1.425)
재미있 (1.288)
최고 (1.119)
즐겁 (0.914)
역시 (0.900)
영상미 (0.747)
도르마무 (0.710)
다음 (0.708)
아이 (0.676)
추천 (0.665)
약간 (0.634)
아이맥스 (0.577)
아쉽 (0.544)
영상 (0.538)
새롭 (0.534)
독특 (0.534)
망토 (0.534)
베니 (0.531)
스트레인 (0.508)
효과 (0.498)
닥터스 (0.486)
호강 (0.465)
연기 (0.452)
귀엽 (0.452)
볼거리 (0.451)
멋지 (0.448)
강추 (0.437)
그리 (0.434)
재미나 (0.420)
한국 (0.418)

부정 단어 상위 30 개
차라리 (-0.561)
개연 (-0.565)
진심 (-0.592)
그나마 (-0.605)
노래 (-0.605)
비추 (-0.607)
스토리 (-0.615)
손예진 (-0.629)
막장 (-0.629)
짬뽕 (-0.667)
보다 (-0.689)
초딩 (-0.738)
정신 (-0.740)
내용 (-0.762)
졸리 (-0.765)
개노잼 (-0.777)
평점 (-0.789)
핵노잼 (-0.817)
쓰레기 (-0.867)
실망 (-0.907)
짜증 (-0.911)
유치 (-0.971)
낭비 (-1.070)
감독 (-1.124)
별로 (-1.296)
알바 (-1.387)
노잼 (-1.555)
아깝 (-1.708)
재미없 (-1.866)
최악 (-1.897)


### TF-IDF 정보 사용하기

In [25]:
lr_tfidf_l2 = LogisticRegression(C=0.1, penalty='l2', solver='saga', max_iter=10000) 
lr_tfidf_l2.fit(tfidf_train_features, train_labels) # 학습
pred_labels_tfidf_l2 = lr_tfidf_l2.predict(tfidf_test_features)

In [26]:
accuracy_score(test_labels, pred_labels_tfidf_l2)

0.9298155737704918

In [27]:
# Get coefficients of the model
coefficients = lr_tfidf_l2.coef_.tolist()

sorted_coefficients = sorted(enumerate(coefficients[0]), key=lambda x:x[1], reverse=True)
# 학습에 사용된 각 단어마다의 coefficient (즉 weight) 값이 존재
# coefficient값이 큰 순으로 정렬 'reverse=True'


K=30
# print top K positive words
print("긍정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[:K]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))
# print top K negative words
print("\n부정 단어 상위 {} 개".format(str(K)))
for word_id, coef in sorted_coefficients[-K:]:
    print('{0:} ({1:.3f})'.format(words_dict[word_id], coef))

긍정 단어 상위 30 개
재밌 (2.279)
재미있 (1.667)
최고 (1.239)
마블 (1.220)
역시 (1.072)
영상미 (0.892)
도르마무 (0.829)
즐겁 (0.823)
아이 (0.803)
영상 (0.678)
추천 (0.660)
다음 (0.643)
기대 (0.611)
아이맥스 (0.492)
베네딕트 (0.475)
베니 (0.454)
새롭 (0.452)
망토 (0.446)
쿠키 (0.429)
닥터스 (0.427)
아쉽 (0.424)
볼거리 (0.424)
약간 (0.424)
효과 (0.423)
멋지 (0.420)
거래 (0.398)
스트레인 (0.395)
닥터 (0.386)
강추 (0.376)
호강 (0.369)

부정 단어 상위 30 개
비밀 (-0.525)
수준 (-0.566)
초딩 (-0.578)
이해 (-0.593)
개연 (-0.599)
개노잼 (-0.613)
노래 (-0.639)
정신 (-0.651)
진심 (-0.660)
실망 (-0.686)
보다 (-0.704)
핵노잼 (-0.739)
그냥 (-0.745)
유치 (-0.791)
쓰레기 (-0.808)
낭비 (-0.808)
음악 (-0.856)
스토리 (-0.864)
짜증 (-0.884)
영화 (-1.017)
내용 (-1.077)
손예진 (-1.165)
노잼 (-1.356)
알바 (-1.488)
별로 (-1.526)
감독 (-1.609)
평점 (-1.785)
최악 (-2.193)
재미없 (-2.209)
아깝 (-2.305)
